In [1]:
# /nafs/dtward/allen/npz_files
# this a npz file
# when you load it you will get a dictionary with two keys
# I (for image)
# this is a 3D array where the first index is cell density, and all the other indices are gene expression level
# so it is 501 x rows x cols big
# the other key is L (for label)
# this is a 1 x rows x cols sized array
# it contains an integer id describing the anatomical structure of interest
# the file is really big and is compressed, so almost certainly you'll need to do some preprocessing to extract small ROIs and save them

#preprocessing step where I choose ROI per slice and write out and save as separate files

#right is 0, left is 1

In [1]:
import os
import sys

path = os.getcwd()
par_path = os.path.abspath(os.pardir)
sys.path.append(par_path)

from utils.data import ExtractROI, AtlasDataset

import matplotlib.pyplot as plt
import numpy as np
import torch

import math
import random as rand
from tqdm import tqdm

from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

In [3]:
ROI_size = (64, 64)
sampling_rate = 0.5
str_sampling_rate = str(sampling_rate).replace('.', '_')

extract_dir = '/nafs/dtward/allen/npz_files/'
save_dir = '/nafs/dtward/allen/rois/'
sub_save_dir = f"{ROI_size[0]}x{ROI_size[1]}_sample_rate_{str_sampling_rate}_center/"
abs_save_dir = os.path.join(save_dir, sub_save_dir)

extractor_center = ExtractROI(extract_dir=extract_dir, save_dir=abs_save_dir)
extractor_center.extract_ROI(ROI_size=ROI_size, sampling_rate=sampling_rate, label_mode='center')

ROI_size = (64, 64)
sampling_rate = 0.5
str_sampling_rate = str(sampling_rate).replace('.', '_')

extract_dir = '/nafs/dtward/allen/npz_files/'
save_dir = '/nafs/dtward/allen/rois/'
sub_save_dir = f"{ROI_size[0]}x{ROI_size[1]}_sample_rate_{str_sampling_rate}_all/"
abs_save_dir = os.path.join(save_dir, sub_save_dir)

extractor_all = ExtractROI(extract_dir=extract_dir, save_dir=abs_save_dir)
extractor_all.extract_ROI(ROI_size=ROI_size, sampling_rate=sampling_rate, label_mode='all')

ROI_size = (64, 64)
sampling_rate = 0.5
str_sampling_rate = str(sampling_rate).replace('.', '_')

extract_dir = '/nafs/dtward/allen/npz_files/'
save_dir = '/nafs/dtward/allen/rois/'
sub_save_dir = f"{ROI_size[0]}x{ROI_size[1]}_sample_rate_{str_sampling_rate}_mode/"
abs_save_dir = os.path.join(save_dir, sub_save_dir)

extractor_all = ExtractROI(extract_dir=extract_dir, save_dir=abs_save_dir)
extractor_all.extract_ROI(ROI_size=ROI_size, sampling_rate=sampling_rate, label_mode='mode')

100%|██████████| 213/213 [07:19<00:00,  2.06s/it]

All ROIs extracted from /nafs/dtward/allen/npz_files/ and saved to /nafs/dtward/allen/rois/64x64_sample_rate_0_5_mode/ under train and test subfolders


In [7]:
train_dir = '/nafs/dtward/allen/rois/64x64_sample_rate_0_5_mode/train/'
test_dir = '/nafs/dtward/allen/rois/64x64_sample_rate_0_5_mode/test/'

train_dataset = AtlasDataset(data_dir=train_dir)
test_dataset = AtlasDataset(data_dir=test_dir)

In [8]:
print(train_dataset[0][0].shape)

(501, 64, 64)


In [13]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [14]:
sample_batch = next(iter(train_loader)) 

In [18]:
print(sample_batch[0].shape)
print(sample_batch[1])
print(len(train_dataset))

torch.Size([32, 501, 64, 64])
tensor([[   0],
        [ 286],
        [   0],
        [   0],
        [   0],
        [ 951],
        [ 951],
        [ 389],
        [   0],
        [   0],
        [ 951],
        [1005],
        [ 593],
        [ 251],
        [ 818],
        [ 997],
        [ 389],
        [   0],
        [1011],
        [ 662],
        [   0],
        [  76],
        [ 194],
        [   0],
        [   0],
        [   0],
        [ 210],
        [   0],
        [   0],
        [   0],
        [   0],
        [   0]])
1590
